In [7]:
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow

In [8]:
from google.colab import files
uploaded = files.upload()

Saving market_data_BIG.xlsx to market_data_BIG (1).xlsx


In [9]:
import pandas as pd

df_big = pd.read_excel("market_data_BIG.xlsx")
# Show first rows
df_big.head()


,Date,EMP,PE,CAPE,DY,Rho,MOV,IR,RR,Y02,...,YSS,NYF,_AU,_DXY,_LCP,_TY,_OIL,_MKT,_VA,_GR
0,1988-04-10,0.020767,12.9,14.469,3.60,-0.083,118.0,6.02,-0.961,7.459,...,0.251622,6.896895,450.50,89.14,2473.7,97.990,16.88,419.06,823.219,462.551
1,1988-04-17,0.020767,12.4,13.960,3.75,-0.078,121.0,5.88,-0.961,7.582,...,0.328976,2.631749,456.25,88.31,2328.2,96.534,18.37,404.66,823.219,462.551
2,1988-04-24,0.020767,12.4,13.950,3.75,-0.051,123.0,5.83,-0.961,7.618,...,0.156475,2.631749,449.25,88.89,2201.4,96.470,18.30,404.71,823.219,462.551
3,1988-05-01,0.020621,12.5,14.036,3.75,-0.054,124.2,5.98,-0.660,7.728,...,0.146747,2.631749,449.00,89.16,2147.8,96.005,18.00,407.55,840.891,461.176
4,1988-05-08,0.020785,12.3,13.761,3.82,-0.079,118.4,6.29,-0.660,7.885,...,0.147628,2.631749,443.00,89.30,2200.9,95.446,17.74,401.29,840.891,461.176


In [31]:
# Delete columns Date and target _MKT
X_full = df_big.drop(columns=["Date", "_MKT"])

# Control: show first rows
print("Shape of dataset used for autoencoder:", X_full.shape)
X_full.head()


Shape of dataset used for autoencoder: (1936, 29)


,EMP,PE,CAPE,DY,Rho,MOV,IR,RR,Y02,Y10,...,DIL,YSS,NYF,_AU,_DXY,_LCP,_TY,_OIL,_VA,_GR
0,0.020767,12.9,14.469,3.60,-0.083,118.0,6.02,-0.961,7.459,8.484,...,0.005760,0.251622,6.896895,450.50,89.14,2473.7,97.990,16.88,823.219,462.551
1,0.020767,12.4,13.960,3.75,-0.078,121.0,5.88,-0.961,7.582,8.737,...,0.005775,0.328976,2.631749,456.25,88.31,2328.2,96.534,18.37,823.219,462.551
2,0.020767,12.4,13.950,3.75,-0.051,123.0,5.83,-0.961,7.618,8.773,...,0.005676,0.156475,2.631749,449.25,88.89,2201.4,96.470,18.30,823.219,462.551
3,0.020621,12.5,14.036,3.75,-0.054,124.2,5.98,-0.660,7.728,8.873,...,0.005315,0.146747,2.631749,449.00,89.16,2147.8,96.005,18.00,840.891,461.176
4,0.020785,12.3,13.761,3.82,-0.079,118.4,6.29,-0.660,7.885,8.990,...,-0.000156,0.147628,2.631749,443.00,89.30,2200.9,95.446,17.74,840.891,461.176


In [11]:
import pandas as pd

df = pd.read_excel("market_data_BIG.xlsx")
df.columns = df.columns.str.strip()  # Remove extra spaces

### 📊 Definizione dei gruppi di segnali (signal sets)

In [12]:
signal_sets = {
    "Set 1: Valuation"        : ['PE', 'CAPE', 'DY'],
    "Set 2: Interest Rates"   : ['IR', 'RR', 'Y02', 'Y10', 'STP'],
    "Set 3: Macro Conditions" : ['GDP', 'M2', 'CPI', 'UN', 'CF'],
    "Set 4: Corporate Health" : ['MG', 'RV', 'ED'],
    "Set 5: Risk Sentiment"   : ['Rho', 'MOV', 'YSS', 'NYF', '_DXY', '_OIL'],
    "Set 6: Style & Asset"    : ['_VA', '_GR', '_AU', '_TY'],
    "Set 7: Full Set"         : [col for col in df.columns if col not in ['Date', '_MKT']]
}

### 🧠 Autoencoder standard applicato a ciascun set di segnali

In [30]:
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
import matplotlib.pyplot as plt

autoencoder_results = {}

for set_name, columns in signal_sets.items():
    print(f"Training AE for: {set_name}")

    # Prepare data
    data = df[columns].dropna()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data)

    # simple AE architecture
    input_dim = X_scaled.shape[1]
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(4, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='linear')(encoded)
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Training
    history = autoencoder.fit(X_scaled, X_scaled, epochs=100, batch_size=16, verbose=0)

    # Save results
    loss = history.history['loss']
    autoencoder_results[set_name] = {
        "loss": loss,
        "final_loss": loss[-1]
    }


Training AE for: Set 1: Valuation
Training AE for: Set 2: Interest Rates
Training AE for: Set 3: Macro Conditions
Training AE for: Set 4: Corporate Health
Training AE for: Set 5: Risk Sentiment
Training AE for: Set 6: Style & Asset
Training AE for: Set 7: Full Set


In [14]:
from keras.models import Model
import numpy as np

latent_codes = {}
target_returns = {}

for set_name, columns in signal_sets.items():
    print(f"Encoding AE for: {set_name}")

    # Prepare data
    data = df[columns + ['_MKT']].dropna()
    X = data[columns].values
    y = data['_MKT'].shift(-1).dropna()  # target

    # Align X e y
    X = X[:-1]
    y = y.values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Architecture
    input_dim = X_scaled.shape[1]
    input_layer = Input(shape=(input_dim,))
    encoded_layer = Dense(4, activation='relu')(input_layer)
    decoded_layer = Dense(input_dim, activation='linear')(encoded_layer)
    autoencoder = Model(inputs=input_layer, outputs=decoded_layer)
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(X_scaled, X_scaled, epochs=100, batch_size=16, verbose=0)

    # Extract latent code
    encoder = Model(inputs=input_layer, outputs=encoded_layer)
    latent = encoder.predict(X_scaled)

    latent_codes[set_name] = latent
    target_returns[set_name] = y


Encoding AE for: Set 1: Valuation
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Encoding AE for: Set 2: Interest Rates
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Encoding AE for: Set 3: Macro Conditions
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Encoding AE for: Set 4: Corporate Health
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Encoding AE for: Set 5: Risk Sentiment
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Encoding AE for: Set 6: Style & Asset
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Encoding AE for: Set 7: Full Set
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [15]:
from scipy.stats import spearmanr

ic_results = {}

for set_name in latent_codes:
    latent = latent_codes[set_name]
    target = target_returns[set_name]

    # IC of each neuron with respect to target
    ic_values = []
    for i in range(latent.shape[1]):
        ic, _ = spearmanr(latent[:, i], target)
        ic_values.append(ic)

    ic_results[set_name] = ic_values
    print(f"{set_name} - Avg IC: {np.mean(ic_values):.4f} | ICs: {np.round(ic_values, 4)}")


Set 1: Valuation - Avg IC: nan | ICs: [-0.274      nan -0.543   0.2183]
Set 2: Interest Rates - Avg IC: -0.3862 | ICs: [-0.6427 -0.5449  0.4846 -0.842 ]
Set 3: Macro Conditions - Avg IC: -0.1236 | ICs: [-0.1963 -0.0797 -0.0081 -0.2102]
Set 4: Corporate Health - Avg IC: 0.1276 | ICs: [ 0.0562 -0.1349  0.3447  0.2441]
Set 5: Risk Sentiment - Avg IC: 0.1988 | ICs: [-0.1927  0.5842 -0.0631  0.4667]
Set 6: Style & Asset - Avg IC: 0.1311 | ICs: [ 0.7936  0.7938 -0.9572 -0.106 ]
Set 7: Full Set - Avg IC: -0.1136 | ICs: [-0.5165 -0.7735  0.6757  0.16  ]


/tmp/ipython-input-15-777778652.py:12: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(latent[:, i], target)


In [16]:
# Simple strategy: long when embedding > 0, short when < 0.
sharpe_results = {}

for set_name in latent_codes:
    latent = latent_codes[set_name]
    target = target_returns[set_name]

    # Simple strategy on first latent neuron
    signal = latent[:, 0]
    positions = np.sign(signal)
    pnl = positions * target

    sharpe = np.mean(pnl) / np.std(pnl) * np.sqrt(252)  # annualized
    sharpe_results[set_name] = sharpe
    print(f"{set_name} - Sharpe Ratio (1st neuron): {sharpe:.4f}")


Set 1: Valuation - Sharpe Ratio (1st neuron): 16.2408
Set 2: Interest Rates - Sharpe Ratio (1st neuron): 16.2182
Set 3: Macro Conditions - Sharpe Ratio (1st neuron): 16.2409
Set 4: Corporate Health - Sharpe Ratio (1st neuron): 8.0673
Set 5: Risk Sentiment - Sharpe Ratio (1st neuron): 16.1556
Set 6: Style & Asset - Sharpe Ratio (1st neuron): 16.2499
Set 7: Full Set - Sharpe Ratio (1st neuron): 15.8500


In [27]:
# Reconstruct results
results = []

for set_name in ic_results:
    avg_ic = np.mean(ic_results[set_name])
    sharpe = sharpe_results[set_name]
    results.append({
        "Signal Set": set_name,
        "Avg IC": round(avg_ic, 4),
        "Avg Sharpe": round(sharpe, 4)
    })

results_df_ae = pd.DataFrame(results).sort_values(
    ["Avg IC", "Avg Sharpe"], ascending=False
)

print(results_df_ae)


                Signal Set  Avg IC  Avg Sharpe
4    Set 5: Risk Sentiment  0.1988     16.1556
5     Set 6: Style & Asset  0.1311     16.2499
3  Set 4: Corporate Health  0.1276      8.0673
6          Set 7: Full Set -0.1136     15.8500
2  Set 3: Macro Conditions -0.1236     16.2409
1    Set 2: Interest Rates -0.3862     16.2182
0         Set 1: Valuation     NaN     16.2408


In [17]:
import tensorflow as tf

from tensorflow.keras.layers import Layer

class ContractiveBottleneck(Layer):
    def __init__(self, units, lam=1e-4, **kwargs):
        super(ContractiveBottleneck, self).__init__(**kwargs)
        self.units = units
        self.lam = lam
        self.dense = Dense(units, activation='sigmoid', name='bottleneck')

    def call(self, inputs):
        h = self.dense(inputs)
        dh = h * (1 - h)  # derivative of sigmoid
        contractive_loss = self.lam * tf.reduce_sum(tf.square(self.dense.kernel)) * tf.reduce_sum(tf.square(dh))
        self.add_loss(contractive_loss)
        return h


In [18]:
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler
import numpy as np

latent_cae = {}
target_returns_cae = {}

for set_name, columns in signal_sets.items():
    print(f"\n🔧 Training Contractive AE for: {set_name}")

    # Prepare data
    data = df[columns + ['_MKT']].dropna()
    X = data[columns].values
    y = data['_MKT'].shift(-1).dropna()
    X = X[:-1]
    y = y.values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # CAE architecture definition
    input_dim = X_scaled.shape[1]
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(12, activation='relu')(input_layer)
    encoded = Dense(6, activation='relu')(encoded)
    bottleneck = ContractiveBottleneck(3)(encoded)
    decoded = Dense(6, activation='relu')(bottleneck)
    decoded = Dense(12, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='linear')(decoded)

    autoencoder_cae = Model(inputs=input_layer, outputs=output_layer)
    autoencoder_cae.compile(optimizer='adam', loss='mse')

    autoencoder_cae.fit(X_scaled, X_scaled, epochs=100, batch_size=16, verbose=0)

    # Extract latent signals
    encoder_cae = Model(inputs=input_layer, outputs=bottleneck)
    latent_cae[set_name] = encoder_cae.predict(X_scaled)
    target_returns_cae[set_name] = y




🔧 Training Contractive AE for: Set 1: Valuation
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🔧 Training Contractive AE for: Set 2: Interest Rates
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🔧 Training Contractive AE for: Set 3: Macro Conditions
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🔧 Training Contractive AE for: Set 4: Corporate Health
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🔧 Training Contractive AE for: Set 5: Risk Sentiment
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🔧 Training Contractive AE for: Set 6: Style & Asset
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

🔧 Training Contractive AE for: Set 7: Full Set
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [20]:
from scipy.stats import spearmanr

# IC computation
ic_results_cae = {}

for set_name in latent_cae:
    latent = latent_cae[set_name]
    target = target_returns_cae[set_name]

    ic_values = []
    for i in range(latent.shape[1]):
        ic, _ = spearmanr(latent[:, i], target)
        ic_values.append(ic)

    ic_results_cae[set_name] = ic_values
    print(f"📉 {set_name} - Avg IC: {np.mean(ic_values):.4f} | ICs: {np.round(ic_values, 4)}")



📉 Set 1: Valuation - Avg IC: -0.0873 | ICs: [-0.3121  0.5272 -0.4769]
📉 Set 2: Interest Rates - Avg IC: 0.2629 | ICs: [-0.1597  0.039   0.9094]
📉 Set 3: Macro Conditions - Avg IC: -0.0014 | ICs: [-0.1117 -0.1631  0.2706]
📉 Set 4: Corporate Health - Avg IC: -0.2978 | ICs: [-0.3207 -0.3164 -0.2564]
📉 Set 5: Risk Sentiment - Avg IC: 0.4516 | ICs: [0.0996 0.6246 0.6306]
📉 Set 6: Style & Asset - Avg IC: 0.0466 | ICs: [ 0.149   0.9119 -0.921 ]
📉 Set 7: Full Set - Avg IC: -0.1863 | ICs: [-0.426   0.7061 -0.839 ]


In [21]:
# Sharpe Ration computation
sharpe_results_cae = {}

for set_name in latent_cae:
    latent = latent_cae[set_name]
    target = target_returns_cae[set_name]

    signal = latent[:, 0]  # first neuron
    positions = np.sign(signal)
    pnl = positions * target

    sharpe = np.mean(pnl) / np.std(pnl) * np.sqrt(252)  # annualized
    sharpe_results_cae[set_name] = sharpe
    print(f"📈 {set_name} - Sharpe Ratio (1st neuron): {sharpe:.4f}")



📈 Set 1: Valuation - Sharpe Ratio (1st neuron): 16.2499
📈 Set 2: Interest Rates - Sharpe Ratio (1st neuron): 16.2499
📈 Set 3: Macro Conditions - Sharpe Ratio (1st neuron): 16.2499
📈 Set 4: Corporate Health - Sharpe Ratio (1st neuron): 16.2499
📈 Set 5: Risk Sentiment - Sharpe Ratio (1st neuron): 16.2499
📈 Set 6: Style & Asset - Sharpe Ratio (1st neuron): 16.2499
📈 Set 7: Full Set - Sharpe Ratio (1st neuron): 16.2499


In [28]:
# Reconstruct results CAE
results_cae = []

for set_name in ic_results_cae:
    avg_ic = np.mean(ic_results_cae[set_name])
    sharpe = sharpe_results_cae[set_name]
    results_cae.append({
        "Signal Set": set_name,
        "Avg IC": round(avg_ic, 4),
        "Avg Sharpe": round(sharpe, 4)
    })

results_df_cae = pd.DataFrame(results_cae).sort_values(
    ["Avg IC", "Avg Sharpe"], ascending=False
)

print(results_df_cae)


                Signal Set  Avg IC  Avg Sharpe
4    Set 5: Risk Sentiment  0.4516     16.2499
1    Set 2: Interest Rates  0.2629     16.2499
5     Set 6: Style & Asset  0.0466     16.2499
2  Set 3: Macro Conditions -0.0014     16.2499
0         Set 1: Valuation -0.0873     16.2499
6          Set 7: Full Set -0.1863     16.2499
3  Set 4: Corporate Health -0.2978     16.2499


In [23]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class VAE(Model):
    def __init__(self, original_dim, latent_dim=3):
        super(VAE, self).__init__()
        self.original_dim = original_dim
        self.latent_dim = latent_dim

        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(original_dim,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(latent_dim + latent_dim),  # z_mean + z_log_var
        ])

        self.decoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(latent_dim,)),
            layers.Dense(32, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(original_dim),
        ])
        self.sampler = Sampling()

    def call(self, inputs):
        z_params = self.encoder(inputs)
        z_mean, z_log_var = tf.split(z_params, num_or_size_splits=2, axis=1)
        z = self.sampler((z_mean, z_log_var))
        reconstruction = self.decoder(z)

        # KL divergence
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
        self.add_loss(tf.reduce_mean(kl_loss))
        return reconstruction


In [24]:
from sklearn.preprocessing import StandardScaler
import numpy as np

latent_vae = {}
target_returns_vae = {}

for set_name, columns in signal_sets.items():
    print(f"\n🚀 Training VAE for: {set_name}")

    # Prepare data
    data = df[columns + ['_MKT']].dropna()
    X = data[columns].values
    y = data['_MKT'].shift(-1).dropna()
    X = X[:-1]
    y = y.values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Create VAE model
    original_dim = X_scaled.shape[1]
    vae = VAE(original_dim=original_dim, latent_dim=3)
    vae.compile(optimizer='adam', loss='mse')

    # VAE training
    vae.fit(X_scaled, X_scaled, epochs=100, batch_size=16, verbose=0)

    # Obtain latent representation (z_mean)
    z_params = vae.encoder.predict(X_scaled)
    z_mean, z_log_var = np.split(z_params, 2, axis=1)

    latent_vae[set_name] = z_mean
    target_returns_vae[set_name] = y




🚀 Training VAE for: Set 1: Valuation


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🚀 Training VAE for: Set 2: Interest Rates


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🚀 Training VAE for: Set 3: Macro Conditions


...

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🚀 Training VAE for: Set 6: Style & Asset


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🚀 Training VAE for: Set 7: Full Set


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [25]:
# IC computation
from scipy.stats import spearmanr

ic_results_vae = {}

for set_name in latent_vae:
    latent = latent_vae[set_name]
    target = target_returns_vae[set_name]

    ic_values = []
    for i in range(latent.shape[1]):
        ic, _ = spearmanr(latent[:, i], target)
        ic_values.append(ic)

    ic_results_vae[set_name] = ic_values
    print(f"📉 {set_name} - Avg IC: {np.mean(ic_values):.4f} | ICs: {np.round(ic_values, 4)}")


📉 Set 1: Valuation - Avg IC: -0.1230 | ICs: [-0.1512  0.3474 -0.5651]
📉 Set 2: Interest Rates - Avg IC: 0.1261 | ICs: [-0.4722  0.4565  0.3939]
📉 Set 3: Macro Conditions - Avg IC: -0.0826 | ICs: [ 0.0787 -0.1168 -0.2098]
📉 Set 4: Corporate Health - Avg IC: -0.0983 | ICs: [-0.1802 -0.1622  0.0475]
📉 Set 5: Risk Sentiment - Avg IC: nan | ICs: [nan nan nan]
📉 Set 6: Style & Asset - Avg IC: 0.4313 | ICs: [-0.6639  0.9768  0.9811]
📉 Set 7: Full Set - Avg IC: -0.0690 | ICs: [-0.1276 -0.1136  0.0344]


/tmp/ipython-input-25-1646413582.py:11: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  ic, _ = spearmanr(latent[:, i], target)


In [26]:
# Sharpe Ration computation
sharpe_results_vae = {}

for set_name in latent_vae:
    latent = latent_vae[set_name]
    target = target_returns_vae[set_name]

    signal = latent[:, 0]  # first neuron
    positions = np.sign(signal)
    pnl = positions * target

    sharpe = np.mean(pnl) / np.std(pnl) * np.sqrt(252)  # annualized
    sharpe_results_vae[set_name] = sharpe
    print(f"📈 {set_name} - Sharpe Ratio (1st neuron): {sharpe:.4f}")


📈 Set 1: Valuation - Sharpe Ratio (1st neuron): -2.1488
📈 Set 2: Interest Rates - Sharpe Ratio (1st neuron): -2.3805
📈 Set 3: Macro Conditions - Sharpe Ratio (1st neuron): -10.0584
📈 Set 4: Corporate Health - Sharpe Ratio (1st neuron): -1.5965
📈 Set 5: Risk Sentiment - Sharpe Ratio (1st neuron): 16.2499
📈 Set 6: Style & Asset - Sharpe Ratio (1st neuron): -5.6411
📈 Set 7: Full Set - Sharpe Ratio (1st neuron): 15.4700


In [29]:
# Reconstruct results VAE
results_vae = []

for set_name in ic_results_vae:
    avg_ic = np.mean(ic_results_vae[set_name])
    sharpe = sharpe_results_vae[set_name]
    results_vae.append({
        "Signal Set": set_name,
        "Avg IC": round(avg_ic, 4),
        "Avg Sharpe": round(sharpe, 4)
    })

results_df_vae = pd.DataFrame(results_vae).sort_values(
    ["Avg IC", "Avg Sharpe"], ascending=False
)

print(results_df_vae)


                Signal Set  Avg IC  Avg Sharpe
5     Set 6: Style & Asset  0.4313     -5.6411
1    Set 2: Interest Rates  0.1261     -2.3805
6          Set 7: Full Set -0.0690     15.4700
2  Set 3: Macro Conditions -0.0826    -10.0584
3  Set 4: Corporate Health -0.0983     -1.5965
0         Set 1: Valuation -0.1230     -2.1488
4    Set 5: Risk Sentiment     NaN     16.2499
